In [10]:
using LinearAlgebra
using TensorKit
using JSON
cd("D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
#push!(LOAD_PATH, "D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
include("kagome_load_tensor.jl")
include("kagome_CTMRG.jl")
include("kagome_model.jl")
include("kagome_IPESS.jl")

D=8
chi=30
tol=1e-6

J1=1;
J2=0;
J3=0;
Jchi=0;
Jtrip=0;

parameters=Dict([("J1", J1), ("J2", J2), ("J3", J3), ("Jchi", Jchi), ("Jtrip", Jtrip)]);

A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);

filenm="LS_D_"*string(D)*"_chi_40.json"
json_dict=read_json_state(filenm);

bond_tensor,triangle_tensor=construct_su2_PG_IPESS(json_dict,A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb);

PEPS_tensor=bond_tensor;
@tensor PEPS_tensor[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];
A_unfused=PEPS_tensor;

U_phy=unitary(fuse(space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7)), space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7));
@tensor A_fused[:] :=PEPS_tensor[-1,-2,-3,-4,1,2,3]*U_phy[-5,1,2,3];

init=Dict([("CTM", []), ("init_type", "PBC")]);
conv_check="singular_value";
@time CTM, AA_fused, U_L,U_D,U_R,U_U=CTMRG(A_fused,chi,conv_check,tol,init);

@time E_up, E_down=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, CTM, "E_triangle");
@time E_up_12, E_up_31, E_up_23, E_down_12, E_down_31, E_down_23=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, CTM, "E_bond");


"initialize CTM"

"start CTM iterations:"

  1.221299 seconds (6.26 M allocations: 732.173 MiB, 17.32% gc time, 19.91% compilation time)
CTMRG iteration: 1, CTMRG err: 4.629281439808446
CTMRG iteration: 2, CTMRG err: 0.27354057170290225
CTMRG iteration: 3, CTMRG err: 0.047370323523708194
CTMRG iteration: 4, CTMRG err: 0.1048347766756141
CTMRG iteration: 5, CTMRG err: 0.02450852047612217
CTMRG iteration: 6, CTMRG err: 0.0991347000340367
CTMRG iteration: 7, CTMRG err: 0.014063717708220173
CTMRG iteration: 8, CTMRG err: 0.003197685223539068
CTMRG iteration: 9, CTMRG err: 0.0009942565565275268
CTMRG iteration: 10, CTMRG err: 0.00038485398554188833
CTMRG iteration: 11, CTMRG err: 0.0001989630396095859
CTMRG iteration: 12, CTMRG err: 0.00011800735276973614
CTMRG iteration: 13, CTMRG err: 7.159031427000397e-5
CTMRG iteration: 14, CTMRG err: 4.261226739771201e-5
CTMRG iteration: 15, CTMRG err: 2.4948822519730957e-5
CTMRG iteration: 16, CTMRG err: 1.4402037424699533e-5
CTMRG iteration: 17, CTMRG err: 8.237803645471843e-6
CTMRG iteration

In [11]:
display((E_up+E_down)/3)

-0.43225342693319496 - 6.92451857959804e-17im

In [13]:
Tleft=CTM["Tset"][4];
Tright=CTM["Tset"][2];


In [14]:
@tensor O1[:]:=Tleft[-3,1,-1]*U_L[1,-2,-4];
@tensor O2[:]:=Tright[-1,1,-3]*U_R[-4,-2,1];


In [15]:
@tensor OO[:]:=O1[-2,-3,-5,1]*O2[-1,1,-4,-6];


In [31]:
U_fuse_chichi=unitary(fuse(space(OO,1)⊗ space(OO,2)),space(OO,1)⊗ space(OO,2));

@tensor OO_fused[:]:=U_fuse_chichi[-1,1,2]*OO[1,2,-2,3,4,-4]*U_fuse_chichi'[3,4,-3];
display(space(OO_fused))

(Rep[SU₂](0=>89, 1/2=>112, 1=>97, 3/2=>48, 2=>9) ⊗ Rep[SU₂](0=>1, 1/2=>2, 1=>1) ⊗ Rep[SU₂](0=>89, 1/2=>112, 1=>97, 3/2=>48, 2=>9)' ⊗ Rep[SU₂](0=>1, 1/2=>2, 1=>1)') ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()

In [122]:
mps_virtual=SU₂Space(0=>1,1/2=>1,1=>1);
mps_phy=space(OO_fused,2);
A = TensorMap(randn, mps_virtual*mps_virtual', mps_phy);
A=permute(A,(1,2,3,),());
display(space(A))


(Rep[SU₂](0=>1, 1/2=>1, 1=>1) ⊗ Rep[SU₂](0=>1, 1/2=>1, 1=>1)' ⊗ Rep[SU₂](0=>1, 1/2=>2, 1=>1)') ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()

In [123]:
@tensor AA[:]:=A[-1,1,-3]*A[1,-2,-4];

@tensor OOA[:]:=OO_fused[-3,1,-4,-5]*A[-1,-2,1];


In [124]:
u,s,v=tsvd(permute(A,(1,),(2,3,)));
display(s)
# for blocks in blocks(s)
#     display(blocks)
# end
# sqrt(s)


TensorMap(ProductSpace(Rep[SU₂](0=>1, 1/2=>1, 1=>1)) ← ProductSpace(Rep[SU₂](0=>1, 1/2=>1, 1=>1))):
* Data for fusiontree FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()) ← FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()):
 2.981277079645829
* Data for fusiontree FusionTree{Irrep[SU₂]}((1/2,), 1/2, (false,), ()) ← FusionTree{Irrep[SU₂]}((1/2,), 1/2, (false,), ()):
 2.47027135698068
* Data for fusiontree FusionTree{Irrep[SU₂]}((1,), 1, (false,), ()) ← FusionTree{Irrep[SU₂]}((1,), 1, (false,), ()):
 2.3116199636652874


In [77]:
function HV_tensor(vl,A)
    @tensor vl[:]:=vl[-1,3,1]*A'[3,-2,2]*A[1,-3,2];
    return vl
end



HV_tensor (generic function with 1 method)

In [125]:
using KrylovKit
f(x)=HV_tensor(x,A);
vl_init = permute(TensorMap(randn, SU₂Space(0=>1), space(A,1)'*space(A,1)), (1,2,3,),());
eu,ev=eigsolve(f, vl_init, 1,:LM,Arnoldi());
display(eu)


3-element Vector{ComplexF64}:
  6.464445062836564 + 0.0im
 3.7100158239340266 + 0.0im
 1.6801732727415875 + 0.0im

In [126]:
using KrylovKit
f(x)=HV_tensor(x,A);
vl_init = permute(TensorMap(randn, SU₂Space(1/2=>1), space(A,1)'*space(A,1)), (1,2,3,),());
eu,ev=eigsolve(f, vl_init, 1,:LM,Arnoldi());
display(eu)


4-element Vector{ComplexF64}:
 3.2851419742204984 + 0.23855927418772074im
 3.2851419742204984 - 0.23855927418772074im
 -2.128290534870929 + 1.2273484179363396im
 -2.128290534870929 - 1.2273484179363396im

In [127]:
using KrylovKit
f(x)=HV_tensor(x,A);
vl_init = permute(TensorMap(randn, SU₂Space(1=>1), space(A,1)'*space(A,1)), (1,2,3,),());
eu,ev=eigsolve(f, vl_init, 1,:LM,Arnoldi());
display(eu)


4-element Vector{ComplexF64}:
    2.629778348343484 + 0.0im
  -0.5143269623953093 + 0.0im
 -0.19571779075763962 + 0.0im
  0.15578935455369203 + 0.0im

In [135]:
A_dense=convert(Array,A);
@tensor AA[:]:=conj(A_dense)[-1,-3,1]*A_dense[-2,-4,1];
AA=reshape(AA,size(AA,1)*size(AA,2),size(AA,3)*size(AA,4));
eigvals(AA)

36-element Vector{ComplexF64}:
  -2.128290534870936 - 1.2273484179363405im
  -2.128290534870936 + 1.2273484179363405im
 -2.1282905348709305 - 1.2273484179363403im
 -2.1282905348709305 + 1.2273484179363403im
 -1.9533793370930312 + 0.0im
 -1.9533793370930235 + 0.0im
 -1.9533793370930232 + 0.0im
 -1.9533793370930232 + 0.0im
 -1.9533793370930224 + 0.0im
 -1.5657044759800969 - 0.4200241266222388im
                     ⋮
  2.6297783483434856 + 0.0im
    2.62977834834349 + 0.0im
  2.6297783483434953 + 0.0im
  3.2851419742205015 - 0.23855927418772183im
  3.2851419742205015 + 0.23855927418772183im
  3.2851419742205037 - 0.23855927418772033im
  3.2851419742205037 + 0.23855927418772033im
   3.710015823934034 + 0.0im
   6.464445062836562 + 0.0im